In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}

table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch7_Attention(스마트번역기)</span>

- Google Neural Machine Translation(GNMT)
- RNN 기반의 Sequence to Sequence 방식
- 인코더(입력) / 디코더(모범출력) 연결 구조
- 응용분야 : 자연어

# 1. 패키지 및 하이퍼파라미터

In [2]:
import numpy as np
import pandas as pd
from time import time
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼파라미터
MY_HIDDEN = 128
MY_EPOCH  = 500

# 2. 학습데이터

In [3]:
raw = pd.read_csv('data/translate.csv',header=None)
# raw.info()
# raw.head()
eng_kor = raw.values.tolist()   # 데이터프레임을 list로 변환 
print(eng_kor[:3])
print('영-한번역 데이터 갯수 : ', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영-한번역 데이터 갯수 :  110


# 3. 영어알파벳과 한글문자 리스트 만들기

In [4]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
print('e_alpha : ',len(e_alpha))
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
print('데이터 한글의 글자수 : ',len(k_ch))
k_ch.sort()
#k_ch

k_alpha = pd.read_csv('data/korean.csv',header=None)[0].tolist()
k_alpha == k_ch # 순서의 내용이 같은지 여부

e_alpha :  29
데이터 한글의 글자수 :  142


True

In [5]:
# cf. 순서 관계없이 내용이 같은지 확인?
from collections import Counter
list1 = [ '가','나','다']
list2 = ['다','나','가']
print(list1==list2)
print(Counter(list1)==Counter(list2))

False
True


In [5]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 :', alpha)
alpha_total_size = len(alpha) # 171 (원핫인코딩 사이드)
print('전체 알파벳 갯수(원핫인코딩 사이즈) : ',alpha_total_size)

영어와 한글 알파벳 : ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩 사이즈) :  171


#  4. 문자당 num를 갖는 dict

In [6]:
char_to_num = {}
for i ,c in enumerate(alpha):
    #print(i,c)
    char_to_num[c] = i
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [7]:
# 리스트컴프리핸션으로
chat_to_num = { c:i for i,c in enumerate(alpha)}
print(chat_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [8]:
# 숫자 = > 문자 / 문자 - > 숫자
print('숫자 - > 문자  :',alpha[5])
print('문자 - > 숫자  :',char_to_num['c'])

숫자 - > 문자  : c
문자 - > 숫자  : 5


In [9]:
data = eng_kor[0]
print('원데이터  : ',data)
# char_to_num['c'],char_to_num['o'],...
print('인코더 입력 (숫자로 전환 전) : ',[ch for ch in data[0]])
print('인코더 입력 : ',[char_to_num[ch] for ch in data[0]])
print('디코더 입력 (숫자로 전환 전) : ',[ch for ch in 'S'+data[1]])
print('디코더 입력 : ',[char_to_num[ch] for ch in 'S'+ data[1]])
print('디코더 출력 : ',[char_to_num[ch] for ch in  data[1]+'E'])

원데이터  :  ['cold', '감기']
인코더 입력 (숫자로 전환 전) :  ['c', 'o', 'l', 'd']
인코더 입력 :  [5, 17, 14, 6]
디코더 입력 (숫자로 전환 전) :  ['S', '감', '기']
디코더 입력 :  [0, 32, 46]
디코더 출력 :  [32, 46, 1]


In [11]:
# 원핫인코딩(방법1)
pd.get_dummies([5,3,7])

,3,5,7
0,0,1,0
1,1,0,0
2,0,0,1


In [12]:
# 원핫인코딩 방법2 : to_catagorical([5,3,7],num_classes=10)
to_categorical([5,3,7],num_classes=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [13]:
# 원학인코딩 방법3 : np.eye(10) 추천
np.eye(10)[[5,3,7]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

# 5. 인코더 입력, 디코더 입력, 디코더 출력

- 디코더출력(원핫인코딩X)

In [10]:
def encoding(eng_kor=eng_kor):
    '인코더입력, 디코더 입력, 디코더 출력 데이터를 return'
    enc_in = []   # 인코더 입력
    dec_in = []   # 디코더 입력
    dec_out = []  # 디코더 출력
    for data in eng_kor:
        # 인코더 입력 데이터 ( 영어 - > 숫자 -> 원핫인코딩)
        eng = [char_to_num[ch] for ch in data[0]]
        #print(data[0],eng)
        #eng_one = to_categorical(eng,num_classes=alpha_total_size)  # alpha_total_size =171
        eng_one =np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)
        #print(enc_in)
        #디코더 입력 데이터 ('S한글' - > 숫자 -> 원핫인코딩)
        kor = [char_to_num[ch] for ch in 'S'+data[1]]
        #print(kor)
        # 원핫인코딩
        kor_one =np.eye(alpha_total_size)[kor]
        #print(kor,'==>  원핫인코딩 하면 : \n',kor_one)
        dec_in.append(kor_one)
        #print(dec_in)
        # 디코더 출력 ( '한글E' => 숫자)
        kor = [char_to_num[ch] for ch in data[1]+'E']
        #print(kor)
        dec_out.append(kor)
    # 인공신경망에 넣을 데이터임으로 numpy 배열로 전환
    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    dec_out = np.array(dec_out)
    #print(enc_in.shape,dec_in.shape,dec_out.shape)
    return enc_in, dec_in, dec_out
sample = [['wood','나무'],['word','단어']]
encoding(sample)

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]),
 array([[[1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]),
 array([[ 48,  83,   1],
        [ 61, 114,   1]]))

In [15]:
# RNN : 타겟변수를 원핫인코딩 (시스템에게 원핫인코딩을 의뢰)
X_enc, X_dec,y_dec = encoding(sample)
X_enc.shape, X_dec.shape,y_dec.shape

((2, 4, 171), (2, 3, 171), (2, 3))

In [16]:
y_dec

array([[ 48,  83,   1],
       [ 61, 114,   1]])

## ※ 축증가

In [17]:
y_dec.reshape(2,3,1) # 방법1

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [18]:
np.expand_dims(y_dec,axis=-1) # 방법2

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [19]:
y_dec[...,np.newaxis]  # 방법3

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [20]:
y_dec[:,:,None] # 방법4

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

# 6. 전체 번역 데이터(독립변수, 타겟변수)

In [11]:
X_enc,X_dec,y_dec = encoding(eng_kor)
Y_dec = np.expand_dims(y_dec,axis=-1)
# Y_dec = y_dec[...,np.newaxis]
# Y_dec = y_dec.reshape(-1,3,1)
X_enc.shape,X_dec.shape,Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

# 7.  Seq2Seq 모델 구현

In [22]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4,alpha_total_size))
# 윗출력, state_c,state_h
_, state_h, state_c = LSTM(
                units=MY_HIDDEN,
                #return_sequences=False, # 윗출력 안받음
                return_state=True,
            )(ENC_IN)
# 인코더와 디코더를 연결할  link
link = [state_h,state_c]
# 디코더 구현 :return_sequences=True 위로 올라가는 출력값 사용
DEC_IN = Input(shape=(3,alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              #return_state=False,
              return_sequences=True)(DEC_IN,
                                     initial_state=link)
# 최종 출력층                                   
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델
model = Model(inputs=[ENC_IN,DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 7-1. Attention 모델 구현

In [12]:
from tensorflow.keras.layers import Attention,Concatenate

In [14]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4,alpha_total_size))
# 윗출력, state_c,state_h
ENC_OUT, state_h, state_c = LSTM(
                units=MY_HIDDEN,
                return_sequences=True, # 윗출력 받음
                return_state=True,
            )(ENC_IN)
# 인코더와 디코더를 연결할  link
link = [state_h,state_c]

# 디코더 구현 :return_sequences=True 위로 올라가는 출력값 사용
DEC_IN = Input(shape=(3,alpha_total_size))
DEC_MID, _, _ = LSTM(units=MY_HIDDEN,
              return_state=True,
              return_sequences=True)(DEC_IN,
                                     initial_state=link)
# 어텐션 메커니즘
CONTEXT_VECTOR = Attention()([DEC_MID,ENC_OUT])

# 컨텍스트벡터와 디코더LSTM 출력을 결합
CONTEXT_AND_LSTM_OUT = Concatenate()([CONTEXT_VECTOR,DEC_MID])

# 최종 출력층                                   
OUT = Dense(units=alpha_total_size,
               activation='softmax')(CONTEXT_AND_LSTM_OUT)

# 모델
model = Model(inputs= [ENC_IN,DEC_IN],
             outputs= OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 4, 128),     153600      ['input_3[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 8. 학습과정 설정 및 학습하기

In [15]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
start = time()
# from tensorflow.keras.callbacks import EarlyStopping
# EarlyStopping=EarlyStopping(monitor='loss',patience=50)
hist = model.fit([X_enc,X_dec],Y_dec,
                epochs=MY_EPOCH,
                 verbose=1)
end = time()
print(f'학습시간 : {end-start}')

Epoch 1/500
4/4 [==============================] - 3s 8ms/step - loss: 5.1153 - accuracy: 0.2091
Epoch 2/500
4/4 [==============================] - 0s 8ms/step - loss: 4.9535 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 7ms/step - loss: 4.1986 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4605 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4026 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3613 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3302 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2997 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2873 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2590 - accuracy: 0.3333
Epoch 11/500
4/4 [===========

4/4 [==============================] - 0s 10ms/step - loss: 0.5540 - accuracy: 0.9576
Epoch 85/500
4/4 [==============================] - 0s 7ms/step - loss: 0.5333 - accuracy: 0.9515
Epoch 86/500
4/4 [==============================] - 0s 7ms/step - loss: 0.5118 - accuracy: 0.9606
Epoch 87/500
4/4 [==============================] - 0s 7ms/step - loss: 0.4924 - accuracy: 0.9515
Epoch 88/500
4/4 [==============================] - 0s 8ms/step - loss: 0.4916 - accuracy: 0.9606
Epoch 89/500
4/4 [==============================] - 0s 7ms/step - loss: 0.4719 - accuracy: 0.9697
Epoch 90/500
4/4 [==============================] - 0s 8ms/step - loss: 0.4292 - accuracy: 0.9697
Epoch 91/500
4/4 [==============================] - 0s 8ms/step - loss: 0.4172 - accuracy: 0.9727
Epoch 92/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3911 - accuracy: 0.9727
Epoch 93/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3770 - accuracy: 0.9788
Epoch 94/500
4/4 [==============

4/4 [==============================] - 0s 7ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0109 - accuracy: 0.9970
Epoch 169/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 174/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0125 - accuracy: 0.9939
Epoch 176/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0123 - accuracy: 0.9970
Epoch 177/500
4/4 [=====

4/4 [==============================] - 0s 7ms/step - loss: 6.9087e-05 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 7ms/step - loss: 6.8534e-05 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 7ms/step - loss: 6.1701e-05 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 7ms/step - loss: 6.1694e-05 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 7ms/step - loss: 7.2040e-05 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 8ms/step - loss: 6.3038e-05 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 7ms/step - loss: 4.9896e-05 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 7ms/step - loss: 4.3636e-05 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1980e-04 - accuracy: 1.0000
Epoch 257/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0026 - accur

4/4 [==============================] - 0s 7ms/step - loss: 2.5175e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 7ms/step - loss: 2.5124e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 7ms/step - loss: 2.5261e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 7ms/step - loss: 2.3018e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 7ms/step - loss: 2.2473e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 7ms/step - loss: 2.2494e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 8ms/step - loss: 2.2307e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 7ms/step - loss: 2.1245e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 8ms/step - loss: 2.0500e-06 - accuracy: 1.0000
Epoch 336/500
4/4 [==============================] - 0s 8ms/step - loss: 2.0074e-06 - a

4/4 [==============================] - 0s 8ms/step - loss: 7.0406e-07 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 7ms/step - loss: 7.0225e-07 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 7ms/step - loss: 6.9177e-07 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 7ms/step - loss: 6.9430e-07 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 7ms/step - loss: 6.7155e-07 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 7ms/step - loss: 6.7624e-07 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 7ms/step - loss: 6.5926e-07 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 8ms/step - loss: 6.6288e-07 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 9ms/step - loss: 6.5746e-07 - accuracy: 1.0000
Epoch 415/500
4/4 [==============================] - 0s 9ms/step - loss: 6.4879e-07 - a

4/4 [==============================] - 0s 8ms/step - loss: 3.9628e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 7ms/step - loss: 3.9628e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 7ms/step - loss: 3.9411e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 8ms/step - loss: 3.8978e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 8ms/step - loss: 3.8725e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 8ms/step - loss: 3.9086e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 11ms/step - loss: 3.8653e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 7ms/step - loss: 3.8147e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 7ms/step - loss: 3.8400e-07 - accuracy: 1.0000
Epoch 494/500
4/4 [==============================] - 0s 7ms/step - loss: 3.7677e-07 - 

In [16]:
model.save('data/attention.h5')

# ※ 12월 5일


In [17]:
from tensorflow.keras.models import load_model
model = load_model('data/attention.h5')

In [18]:
# 쉬운문제
easy_test = [['find','PP'],
            ['cold','PP'],
            ['cook','PP'],
            ['date','PP'],
            ['desk','PP']]
enc_in,dec_in,dec_out = encoding(easy_test)
enc_in.shape,dec_in.shape

((5, 4, 171), (5, 3, 171))

In [19]:
pred = model.predict([enc_in,dec_in])
pred.shape

1/1 [==============================] - 1s 792ms/step


(5, 3, 171)

In [20]:
#pred[0]
for i in range(len(pred)):
    eng =easy_test[i][0]
    hat = pred[i].argmax(axis=-1)[:-1]
    #kor = alpha[hat[0]] + alpha[hat[1]]
    kor = ''.join([alpha[num] for num in hat])
    print("{} = >{} {}".format(eng,kor,hat))
   

find = >찾다 [148  60]
cold = >감기 [32 46]
cook = >요리 [122  72]
date = >날짜 [ 49 146]
desk = >책상 [149  97]


In [21]:
# 어려운 문제
hard_test = [['love','PP'],
             ['lvoe','PP'],
             ['loev','PP'],
             ['olve','PP'],
             ['loev','PP'],
             ['elov','PP'],
             ['evol','PP'],
             ['rave','PP'],]

In [22]:
enc_in,dec_in, _ = encoding(hard_test)
enc_in.shape,dec_in.shape

((8, 4, 171), (8, 3, 171))

In [23]:
pred = model.predict([enc_in,dec_in],verbose=0).argmax(axis=-1)
pred

array([[ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [ 60,   1,   1],
       [ 75,  60,   1],
       [136, 140,   1]], dtype=int64)

In [24]:
for i in range(len(pred)):
    eng = hard_test[i][0]
    kor = ''.join([alpha[num] for num in pred[i]])
    print("{} =>{}{}".format(eng,kor[:-1],pred[i][:-1]))

love =>사랑[96 66]
lvoe =>사랑[96 66]
loev =>사랑[96 66]
olve =>사랑[96 66]
loev =>사랑[96 66]
elov =>다E[60  1]
evol =>매다[75 60]
rave =>읽장[136 140]
